# Preferential Bayesian Optimization: EI
This notebook demonstrates the use of the Expected Improvement (EI) acquisition function on ordinal (preference) data.

Formulation by Nguyen Quoc Phong.

In [1]:
import numpy as np
import gpflow
import tensorflow as tf
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
import sys
import os
import pickle

from gpflow.utilities import set_trainable, print_summary
gpflow.config.set_default_summary_fmt("notebook")

sys.path.append(os.path.split(os.path.split(os.path.split(os.getcwd())[0])[0])[0]) # Move 3 levels up directory to import PBO
import PBO

In [2]:
gpu_to_use = 0

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only use the first GPU
    try:
        for gpu in gpus:
              tf.config.experimental.set_memory_growth(gpu, True)
        tf.config.experimental.set_visible_devices(gpus[gpu_to_use], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)

Num GPUs Available:  5
5 Physical GPUs, 1 Logical GPU


In [3]:
lengthscale = 0.3
lengthscale_prior_alpha = tf.constant(2, dtype=tf.float64)
lengthscale_prior_beta = tf.constant(4, dtype=tf.float64)

In [4]:
objective = PBO.objectives.hartmann3d
objective_low = 0
objective_high = 1.
objective_name = "Hart3"
acquisition_name = "EI"
experiment_name = "PBO" + "_" + acquisition_name + "_" + objective_name + "FullGP" + "v2"

In [5]:
num_runs = 10
num_evals = 50
num_samples = 1000
num_choices = 2
input_dims = 3
objective_dim = input_dims # CHANGE 1: require the objective dim
num_maximizers = 20
num_init_prefs = 12 # CHANGE 2: randomly initialize with some preferences

# CHANGE 1: reduce the value of delta to avoid numerical error
# as k(x,x') = sigma^2 * exp( -[(x-x')/l]^2 )
# which could be very small if l is too small
# so we define l relatively by the range of input (objective_high - objective_low)
#   It is ok for the total number of observations > the total number of possible inputs
# because there is a noise in the observation, it might require repeated observations 
# at the same input pair to improve the confidence 
num_discrete_per_dim = 20
delta = (objective_high - objective_low) / num_discrete_per_dim

In [6]:
results_dir = os.getcwd() + '/results/' + experiment_name + '/'

try:
    # Create target Directory
    os.makedirs(results_dir)
    print("Directory " , results_dir ,  " created ") 
except FileExistsError:
    print("Directory " , results_dir ,  " already exists")

Directory  /home/sebtsh/PBO/notebooks/EI/results/PBO_EI_Hart3FullGPv2/  already exists


In [7]:
def get_noisy_observation(X, objective):
    f = PBO.objectives.objective_get_f_neg(X, objective)
    return PBO.observation_model.gen_observation_from_f(X, f, 1)

In [8]:
def train_and_visualize(X, y, title, lengthscale_init=None, signal_variance_init=None):
    lengthscale_prior = tfp.distributions.Gamma(concentration=lengthscale_prior_alpha,
                                               rate=lengthscale_prior_beta)
    
    # Train model with data
    # CHANGE 6: use full_gp instead of sparse, 
    result = PBO.models.learning_fullgp.train_model_fullcov(
                        X, y, 
                        obj_low=objective_low,
                        obj_high=objective_high,
                        lengthscale_init=lengthscale_init,
                        signal_variance_init=signal_variance_init,
                        indifference_threshold=0.,
                        n_sample=1000,
                        deterministic=True, # only sample f values once, not re-sampling
                        num_steps=3000)
    
    q_mu = result['q_mu']
    q_sqrt = result['q_sqrt']
    u = result['u']
    inputs = result['inputs']
    k = result['kernel']
    
    likelihood = gpflow.likelihoods.Gaussian()
    model = PBO.models.learning.init_SVGP_fullcov(q_mu, q_sqrt, u, k, likelihood)
    u_mean = q_mu.numpy()
    inducing_vars = u.numpy()
    
    return model, inputs, u_mean, inducing_vars

In [9]:
def uniform_grid(input_dims, num_discrete_per_dim, low=0., high=1.):
    """
    Returns an array with all possible permutations of discrete values in input_dims number of dimensions.
    :param input_dims: int
    :param num_discrete_per_dim: int
    :param low: int
    :param high: int
    :return: tensor of shape (num_discrete_per_dim ** input_dims, input_dims)
    """
    num_points = num_discrete_per_dim ** input_dims
    out = np.zeros([num_points, input_dims])
    discrete_points = np.linspace(low, high, num_discrete_per_dim)
    for i in range(num_points):
        for dim in range(input_dims):
            val = num_discrete_per_dim ** (dim)
            out[i, dim] = discrete_points[int((i // val) % num_discrete_per_dim)]
    return out

This function is our main metric for the performance of the acquisition function: The closer the model's best guess to the global minimum, the better.

In [10]:
def best_guess(model):
    """
    Returns a GP model's best guess of the global maximum of f.
    """
    # CHANGE 7: use a discrete grid
    xx = PBO.models.learning_fullgp.get_all_discrete_inputs(objective_low, objective_high, objective_dim, delta)
    res = model.predict_f(xx)[0].numpy()
    return xx[np.argmax(res)]

Store the results in these arrays:

In [11]:
num_data_at_end = int(num_init_prefs + num_evals)
X_results = np.zeros([num_runs, num_data_at_end, num_choices, input_dims])
y_results = np.zeros([num_runs, num_data_at_end, 1, input_dims])
best_guess_results = np.zeros([num_runs, num_evals, input_dims])

Create the initial values for each run:

In [12]:
np.random.seed(0)

# CHANGE 8: just randomly initialize with some preference observation
init_vals = np.zeros([num_runs, num_init_prefs, num_choices, input_dims])

for run in range(num_runs):
    for i in range(num_init_prefs):
        init_vals[run,i] = PBO.models.learning_fullgp.get_random_inputs(
                                objective_low, 
                                objective_high, 
                                objective_dim, 
                                delta,
                                size=num_choices,
                                with_replacement=False,
                                exclude_inputs=None)

The following loops carry out the Bayesian optimization algorithm over a number of runs, with a fixed number of evaluations per run.

In [ ]:
# CHANGE 9: need to store lengthscale and signal_variance from previous iteration to initialize the current iteration
lengthscale_init = None
signal_variance_init = None

for run in range(num_runs):  # CHECK IF STARTING RUN IS CORRECT
    print("")
    print("==================")
    print("Beginning run %s" % (run))
    
    X = init_vals[run]
    y = get_noisy_observation(X, objective)
    
    model, inputs, u_mean, inducing_vars = train_and_visualize(X, y, 
                                                        "Run_{}:_Initial_model".format(run))
    # save optimized lengthscale and signal variance for next iteration
    lengthscale_init = model.kernel.lengthscale.numpy()
    signal_variance_init = model.kernel.variance.numpy()
    
    for evaluation in range(num_evals):
        print("Beginning evaluation %s" % (evaluation)) 

        # Get incumbent maximizer
        input_vals = model.predict_f(inputs)[0].numpy()
        maximizer = np.expand_dims(inputs[np.argmax(input_vals)], axis=0)
        
        print("Maximizer:")
        print(maximizer)
        
        # Sample possible next input points. In EI, all queries are a pair with the incumbent maximizer as the 
        # first point and a next input point as the second point
        
        samples = PBO.models.learning_fullgp.get_random_inputs(low=objective_low,
                                                               high=objective_high,
                                                               dim=objective_dim,
                                                               delta=delta,
                                                               size=num_samples,
                                                               exclude_inputs=maximizer)

        
        # Calculate EI vals
        ei_vals = PBO.acquisitions.ei.EI(model, maximizer, samples)
        
        L = np.argsort(np.ravel(-ei_vals))  # n-th element in this (num_samples, ) size array is the index of n-th
        #largest element in ei_vals
        
        # Select query that maximizes EI
        if np.all(np.equal(samples[L[0]], maximizer)):  #if value with highest EI is same as maximizer, pick the next
            # highest value. Else pick this
            next_idx = L[1]
        else:
            next_idx = L[0]
            
        next_query = np.zeros((num_choices, input_dims))
        next_query[0, :] = maximizer  # EI only works in binary choices
        next_query[1, :] = samples[next_idx]
        print("Evaluation %s: Next query is %s with EI value of %s" % (evaluation, next_query, ei_vals[next_idx]))

        X = np.concatenate([X, [next_query]])
        # Evaluate objective function
        y = np.concatenate([y, get_noisy_observation(np.expand_dims(next_query, axis=0), objective)], axis=0)
        
        print("Evaluation %s: Training model" % (evaluation))
        model, inputs, u_mean, inducing_vars = train_and_visualize(X, y,  
                                                                   "Run_{}_Evaluation_{}".format(run, evaluation))
        
        print_summary(model)

        # save optimized lengthscale and signal variance for next iteration
        lengthscale_init = model.kernel.lengthscale.numpy()
        signal_variance_init = model.kernel.variance.numpy()

        best_guess_results[run, evaluation, :] = best_guess(model)
        # CHANGE 11: log both the estimated minimizer and its objective value
        print("Best_guess f({}) = {}".format(
                best_guess_results[run, evaluation, :], 
                objective(best_guess_results[run, evaluation, :])))
        
        # Save model
        pickle.dump((X, y, inputs, 
                     model.kernel.variance, 
                     model.kernel.lengthscale, 
                     model.likelihood.variance, 
                     inducing_vars, 
                     model.q_mu, 
                     model.q_sqrt, 
                     maximizer), 
                    open(results_dir + "Model_Run_{}_Evaluation_{}.p".format(run, evaluation), "wb"))

    X_results[run] = X
    y_results[run] = y


Beginning run 5
Optimizer config:  {'name': 'RMSprop', 'learning_rate': 0.001, 'decay': 0.0, 'rho': 0.0, 'momentum': 0.0, 'epsilon': 1e-07, 'centered': False}
Indifference_threshold is fixed at 0.0
Initialize lengthscale at [0.5 0.5 0.5]
       signal variance at 1.0
   Initial negative ELBO: 10691.447565168432
Negative ELBO at step 0: 10564.312646318058 in 0.9495s
Negative ELBO at step 1000: 10.755720002407335 in 78.9430s
Negative ELBO at step 1500: 10.44127202053768 in 75.2038s
Negative ELBO at step 2000: 10.280580723755183 in 74.3859s
Negative ELBO at step 2500: 10.175633147479418 in 71.8754s
Beginning evaluation 0
Maximizer:
[[0.575 0.475 0.725]]
Evaluation 0: Next query is [[0.575 0.475 0.725]
 [0.475 0.525 0.725]] with EI value of [0.48373297]
Evaluation 0: Training model
Optimizer config:  {'name': 'RMSprop', 'learning_rate': 0.001, 'decay': 0.0, 'rho': 0.0, 'momentum': 0.0, 'epsilon': 1e-07, 'centered': False}
Indifference_threshold is fixed at 0.0
Initialize lengthscale at [0

name,class,transform,prior,trainable,shape,dtype,value
SVGP.kernel.variance,Parameter,Softplus,,True,(),float64,1.4905568000371512
SVGP.kernel.lengthscale,Parameter,Softplus + Shift,,True,"(3,)",float64,[0.24299694 0.2434291 0.23034812]
SVGP.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,1.0
SVGP.inducing_variable.Z,Parameter,,,False,"(25, 3)",float64,"[[0.525, 0.025, 0.525..."
SVGP.q_mu,Parameter,,,False,"(25, 1)",float64,[[-0.33801582...
SVGP.q_sqrt,Parameter,FillTriangular,,False,"(1, 25, 25)",float64,"[[[1.00799547e+00, 0.00000000e+00, 0.00000000e+00..."


Best_guess f([0.325 0.525 0.675]) = -2.1144269105565723
Beginning evaluation 1
Maximizer:
[[0.475 0.525 0.725]]
Evaluation 1: Next query is [[0.475 0.525 0.725]
 [0.325 0.475 0.725]] with EI value of [0.45069687]
Evaluation 1: Training model
Optimizer config:  {'name': 'RMSprop', 'learning_rate': 0.001, 'decay': 0.0, 'rho': 0.0, 'momentum': 0.0, 'epsilon': 1e-07, 'centered': False}
Indifference_threshold is fixed at 0.0
Initialize lengthscale at [0.5 0.5 0.5]
       signal variance at 1.0
   Initial negative ELBO: 27098.313230705102
Negative ELBO at step 0: 26754.363597582786 in 0.2775s
Negative ELBO at step 500: 128.54468013752654 in 84.0565s
Negative ELBO at step 1000: 12.295805839685372 in 83.1344s
Negative ELBO at step 1500: 12.016113362561484 in 82.9031s
Negative ELBO at step 2000: 12.016096318601159 in 81.6484s
Negative ELBO at step 2500: 12.016081655641647 in 82.7805s


name,class,transform,prior,trainable,shape,dtype,value
SVGP.kernel.variance,Parameter,Softplus,,True,(),float64,1.4766317469790917
SVGP.kernel.lengthscale,Parameter,Softplus + Shift,,True,"(3,)",float64,[0.30191084 0.25220819 0.22066389]
SVGP.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,1.0
SVGP.inducing_variable.Z,Parameter,,,False,"(26, 3)",float64,"[[0.525, 0.025, 0.525..."
SVGP.q_mu,Parameter,,,False,"(26, 1)",float64,[[-0.31799802...
SVGP.q_sqrt,Parameter,FillTriangular,,False,"(1, 26, 26)",float64,"[[[9.90002171e-01, 0.00000000e+00, 0.00000000e+00..."


Best_guess f([0.225 0.475 0.675]) = -2.003432707983793
Beginning evaluation 2
Maximizer:
[[0.325 0.475 0.725]]
Evaluation 2: Next query is [[0.325 0.475 0.725]
 [0.125 0.425 0.725]] with EI value of [0.41418058]
Evaluation 2: Training model
Optimizer config:  {'name': 'RMSprop', 'learning_rate': 0.001, 'decay': 0.0, 'rho': 0.0, 'momentum': 0.0, 'epsilon': 1e-07, 'centered': False}
Indifference_threshold is fixed at 0.0
Initialize lengthscale at [0.5 0.5 0.5]
       signal variance at 1.0
   Initial negative ELBO: 35018.09420582855
Negative ELBO at step 0: 34571.458094975394 in 0.2708s
Negative ELBO at step 500: 142.7338116533183 in 86.9867s
Negative ELBO at step 1000: 13.042125199564026 in 86.8570s
Negative ELBO at step 1500: 12.68685775960957 in 86.0538s
Negative ELBO at step 2000: 12.686845489312578 in 87.2239s
Negative ELBO at step 2500: 12.686834842871972 in 88.7741s


...

Best_guess f([0.325 0.625 0.825]) = -3.593266574210071
Beginning evaluation 42
Maximizer:
[[0.325 0.625 0.825]]
Evaluation 42: Next query is [[0.325 0.625 0.825]
 [0.025 0.625 0.675]] with EI value of [0.0646468]
Evaluation 42: Training model
Optimizer config:  {'name': 'RMSprop', 'learning_rate': 0.001, 'decay': 0.0, 'rho': 0.0, 'momentum': 0.0, 'epsilon': 1e-07, 'centered': False}
Indifference_threshold is fixed at 0.0
Initialize lengthscale at [0.32 0.32 0.32]
       signal variance at 1.0
   Initial negative ELBO: 1433411988.402114
Negative ELBO at step 0: 1401449607.4039755 in 1.0483s
Negative ELBO at step 500: 430022.5047145428 in 259.2711s
Negative ELBO at step 1000: 93.56043302980548 in 259.9579s
Negative ELBO at step 1500: 44.8490789513598 in 262.7765s
Negative ELBO at step 2000: 41.52999647625069 in 261.6257s
Negative ELBO at step 2500: 39.42620816850792 in 264.0935s


name,class,transform,prior,trainable,shape,dtype,value
SVGP.kernel.variance,Parameter,Softplus,,True,(),float64,0.7232493015448822
SVGP.kernel.lengthscale,Parameter,Softplus + Shift,,True,"(3,)",float64,[0.05277794 0.01860296 0.06832259]
SVGP.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,1.0
SVGP.inducing_variable.Z,Parameter,,,False,"(54, 3)",float64,"[[0.725, 0.625, 0.025..."
SVGP.q_mu,Parameter,,,False,"(54, 1)",float64,[[0.27764512...
SVGP.q_sqrt,Parameter,FillTriangular,,False,"(1, 54, 54)",float64,"[[[8.13722007e-01, 0.00000000e+00, 0.00000000e+00..."


Best_guess f([0.325 0.625 0.825]) = -3.593266574210071
Beginning evaluation 43
Maximizer:
[[0.325 0.625 0.825]]
Evaluation 43: Next query is [[0.325 0.625 0.825]
 [0.075 0.625 0.725]] with EI value of [0.08015777]
Evaluation 43: Training model
Optimizer config:  {'name': 'RMSprop', 'learning_rate': 0.001, 'decay': 0.0, 'rho': 0.0, 'momentum': 0.0, 'epsilon': 1e-07, 'centered': False}
Indifference_threshold is fixed at 0.0
Initialize lengthscale at [0.32 0.32 0.32]
       signal variance at 1.0
   Initial negative ELBO: 1570876515.5165942
Negative ELBO at step 0: 1536648207.3450375 in 1.0497s
Negative ELBO at step 500: 451415.0314718991 in 267.3479s
Negative ELBO at step 1000: 91.48453522776192 in 266.1254s
Negative ELBO at step 1500: 45.53153509047183 in 267.0873s
Negative ELBO at step 2000: 42.26539300684522 in 263.8796s
Negative ELBO at step 2500: 40.02393348000654 in 264.7629s


name,class,transform,prior,trainable,shape,dtype,value
SVGP.kernel.variance,Parameter,Softplus,,True,(),float64,0.7253109028758641
SVGP.kernel.lengthscale,Parameter,Softplus + Shift,,True,"(3,)",float64,[0.05562464 0.01860297 0.06898585]
SVGP.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,1.0
SVGP.inducing_variable.Z,Parameter,,,False,"(54, 3)",float64,"[[0.725, 0.625, 0.025..."
SVGP.q_mu,Parameter,,,False,"(54, 1)",float64,[[0.30756081...
SVGP.q_sqrt,Parameter,FillTriangular,,False,"(1, 54, 54)",float64,"[[[8.14028713e-01, 0.00000000e+00, 0.00000000e+00..."


Best_guess f([0.325 0.625 0.825]) = -3.593266574210071
Beginning evaluation 44
Maximizer:
[[0.325 0.625 0.825]]
Evaluation 44: Next query is [[0.325 0.625 0.825]
 [0.275 0.625 0.775]] with EI value of [0.06256924]
Evaluation 44: Training model
Optimizer config:  {'name': 'RMSprop', 'learning_rate': 0.001, 'decay': 0.0, 'rho': 0.0, 'momentum': 0.0, 'epsilon': 1e-07, 'centered': False}
Indifference_threshold is fixed at 0.0
Initialize lengthscale at [0.32 0.32 0.32]
       signal variance at 1.0
   Initial negative ELBO: 1538159851.6248748
Negative ELBO at step 0: 1504148294.1801772 in 1.0424s


In [ ]:
pickle.dump((X_results, y_results, best_guess_results), 
            open(results_dir + acquisition_name + "_" + objective_name + "_" + "Xybestguess.p", "wb"))

In [ ]:
def dist(x, y):
    """
    x and y have shape (..., input_dims)
    """
    return np.sqrt(np.sum((x - y) * (x - y), axis=-1))

xx = uniform_grid(input_dims, num_discrete_per_dim, low=objective_low, high=objective_high)
global_min = xx[np.argmin(objective(xx))][0]

for i in range(best_guess_results.shape[0]):
    diff_from_min = dist(best_guess_results[i], global_min)
    
    x_axis = list(range(num_combs+1, num_combs+1+num_evals))
    plt.figure(figsize=(12, 6))
    plt.plot(x_axis, diff_from_min, 'kx', mew=2)
    plt.xticks(x_axis)
    plt.xlabel('Evaluations', fontsize=18)
    plt.ylabel('Best guess distance', fontsize=16)
    plt.title("Run %s" % i)
    plt.show()